In [ ]:
# https://github.com/harveyp123/DPR-hongwu/blob/main/my_DPR_try/test_vllm_mixtral_MoE.ipynb
# https://github.com/harveyp123/DPR-hongwu/tree/main

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3" # define GPU id, remove if you want to use all GPUs available
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

In [2]:
num_gpus = 4
# model = "HuggingFaceH4/zephyr-7b-beta"
# mode = "mistralai/Mixtral-8x7B-v0.1"
model = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"

llm = LLM(model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO", dtype="bfloat16", gpu_memory_utilization=0.96, tensor_parallel_size = num_gpus, download_dir = '../models_pretrain/')
tokenizer = AutoTokenizer.from_pretrained(model, cache_dir = '../models_pretrain/')

2024-04-10 21:08:20,581	INFO worker.py:1752 -- Started a local Ray instance.


INFO 04-10 21:08:21 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO', tokenizer='NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir='../models_pretrain/', load_format=auto, tensor_parallel_size=4, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(RayWorkerVllm pid=3993788) INFO 04-10 21:08:33 selector.py:16] Using FlashAttention backend.
INFO 04-10 21:08:33 selector.py:16] Using FlashAttention backend.
(RayWorkerVllm pid=3994131) INFO 04-10 21:08:34 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 04-10 21:08:35 pynccl_utils.py:45] vLLM is using nccl==2.18.1
(RayWorkerVllm pid=3993788) INFO 04-10 21:08:47 weight_utils.py:177] Using model weights format ['*.safetensors']
(RayWorkerVllm pid=3994028) INFO 04-10 21:08:33 selector.py:16] Using FlashAttention backend. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayWorkerVllm pid=3994028) INFO 04-10 21:08:35 pynccl_utils.py:45] vLLM is using nccl==2.18.1 [repeated 2x across cluster]
INFO 04-10 21:08:47 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-10 21:08:53 model_runner.py:

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generation using base model

In [6]:
#### Base model generation
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens= 20, ignore_eos=False)
prompt = "How are you today? "
outputs = llm.generate(prompt, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s]

Prompt: 'How are you today? ', Generated text: '🙂\n\nI want to share with you a story from my personal life.\n\nI spent'


Generation using chat model

In [4]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens= 200, ignore_eos=False)

messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in a polite fashion",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
 ]

#### Reference: https://huggingface.co/docs/transformers/main/chat_templating
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)
outputs = llm.generate(prompt, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: \n{prompt!r} \n Generated text: \n{generated_text!r}")

<|im_start|>system
You are a friendly chatbot who always responds in a polite fashion<|im_end|>
<|im_start|>user
How many helicopters can a human eat in one sitting?<|im_end|>
<|im_start|>assistant



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]

Prompt: 
'<|im_start|>system\nYou are a friendly chatbot who always responds in a polite fashion<|im_end|>\n<|im_start|>user\nHow many helicopters can a human eat in one sitting?<|im_end|>\n<|im_start|>assistant\n' 
 Generated text: 
'I would say that humans are not physiologically designed to consume helicopters in any sitting. Helicopters are large, complex machines made of metal and other materials, which are not edible. Please remember to stick to a healthy and balanced diet.'
